In [2]:
from graphframes import *

READING CSV FILES AND REGISTERING TEMPLATE

In [3]:
stations = sqlContext.read.format("com.databricks.spark.csv").options(header='true', inferschema='true').load("file:///home/cloudera/Downloads/BDM1 Homework Assignment-1-20180521/station.csv")

In [4]:
stations.registerTempTable("stations")


In [5]:
trips = sqlContext.read.format("com.databricks.spark.csv").options(header='true').load("file:///home/cloudera/Downloads/BDM1 Homework Assignment-1-20180521/trip.csv")

In [6]:
trips.registerTempTable("trips")

CHANGING FIELD NAMES TO MAKE GRAPHFRAMES

In [7]:
stations = stations.withColumnRenamed("name", "id").distinct()

In [8]:
trips = trips.withColumnRenamed("Start Station", "src").distinct()

In [9]:
trips = trips.withColumnRenamed("End Station", "dst").distinct()

CHECKING FOR COLUMN NAMES

In [122]:
stations.columns

['station_id', 'id', 'lat', 'long', 'dockcount', 'landmark', 'installation']

In [107]:
trips.cache()
stations.cache()

DataFrame[station_id: int, id: string, lat: double, long: double, dockcount: int, src: string, installation: string]

GREATING GRAPH

In [10]:
g = GraphFrame(stations,trips)
print g


GraphFrame(v:[id: string, station_id: int, lat: double, long: double, dockcount: int, landmark: string, installation: string], e:[src: string, dst: string, Trip ID: string, Duration: string, Start Date: string, Start Terminal: string, End Date: string, End Terminal: string, Bike #: string, Subscriber Type: string, Zip Code: string])


In [124]:
g.vertices.show()

+----------+--------------------+---------+-----------+---------+-------------+------------+
|station_id|                  id|      lat|       long|dockcount|     landmark|installation|
+----------+--------------------+---------+-----------+---------+-------------+------------+
|         8| San Salvador at 1st|37.330165|-121.885831|       15|     San Jose|    8/5/2013|
|        12|SJSU 4th at San C...|37.332808|-121.883891|       19|     San Jose|    8/7/2013|
|        73|Grant Avenue at C...|37.798522|-122.407245|       15|San Francisco|   8/21/2013|
|        36|California Ave Ca...|37.429082|-122.142805|       15|    Palo Alto|   8/14/2013|
|        50|Harry Bridges Pla...|37.795392|-122.394203|       23|San Francisco|   8/20/2013|
|        27|Mountain View Cit...|37.389218|-122.081896|       15|Mountain View|   8/16/2013|
|        32|Castro Street and...|37.385956|-122.083678|       11|Mountain View|  12/31/2013|
|        59| Golden Gate at Polk|37.781332|-122.418603|       23|San F

In [125]:
g.edges.show()

+-------+--------+---------------+--------------------+--------------+---------------+--------------------+------------+------+---------------+--------+
|Trip ID|Duration|     Start Date|                 src|Start Terminal|       End Date|                 dst|End Terminal|Bike #|Subscriber Type|Zip Code|
+-------+--------+---------------+--------------------+--------------+---------------+--------------------+------------+------+---------------+--------+
| 912665|     824|8/31/2015 16:14|Embarcadero at Sa...|            60|8/31/2015 16:28|Harry Bridges Pla...|          50|   511|       Customer|     nil|
| 912486|    1033|8/31/2015 13:20|      Market at 10th|            67|8/31/2015 13:37|   2nd at South Park|          64|   111|     Subscriber|   94552|
| 912365|     570|8/31/2015 10:51|Embarcadero at Br...|            54|8/31/2015 11:00|     Clay at Battery|          41|   377|     Subscriber|   94105|
| 912092|     232| 8/31/2015 8:56|San Francisco Cal...|            70| 8/31/2015 9

SHOWING INDEGREES DESCENING 

In [158]:
# g.inDegrees.show()
g.inDegrees.sort("inDegree",ascending =False).limit(20).show()

+--------------------+--------+
|                  id|inDegree|
+--------------------+--------+
|San Francisco Cal...|   34810|
|San Francisco Cal...|   22523|
|Harry Bridges Pla...|   17810|
|     2nd at Townsend|   15463|
|     Townsend at 7th|   15422|
|Embarcadero at Sa...|   15065|
|   Market at Sansome|   13916|
|   Steuart at Market|   13617|
|Temporary Transba...|   12966|
|  Powell Street BART|   10239|
|      Market at 10th|   10220|
|       Market at 4th|    9685|
|   2nd at South Park|    8253|
|       5th at Howard|    8147|
|Civic Center BART...|    7714|
|       Howard at 2nd|    7275|
|Embarcadero at Fo...|    7229|
|Broadway St at Ba...|    7159|
|Embarcadero at Br...|    6687|
|     Beale at Market|    6330|
+--------------------+--------+



SHOWING OUTDEGREES DESCENDING

In [160]:
g.outDegrees.sort("outDegree",ascending =False).limit(20).show()

+--------------------+---------+
|                  id|outDegree|
+--------------------+---------+
|San Francisco Cal...|    26304|
|San Francisco Cal...|    21758|
|Harry Bridges Pla...|    17255|
|Temporary Transba...|    14436|
|Embarcadero at Sa...|    14158|
|     2nd at Townsend|    14026|
|     Townsend at 7th|    13752|
|   Steuart at Market|    13687|
|      Market at 10th|    11885|
|   Market at Sansome|    11431|
|       Market at 4th|     9894|
|  Powell Street BART|     9695|
|   2nd at South Park|     9469|
|     Beale at Market|     8359|
|Grant Avenue at C...|     8337|
|       2nd at Folsom|     7999|
|Civic Center BART...|     7760|
|       5th at Howard|     7708|
|Broadway St at Ba...|     7676|
|Embarcadero at Fo...|     7596|
+--------------------+---------+



In [ ]:
STATIONS WITH NO CONNECTING FLIGHTS

In [172]:
motifs = g.find("(a)-[]->(b)")
motifs.show(10)

+--------------------+--------------------+
|                   a|                   b|
+--------------------+--------------------+
|[60,Embarcadero a...|[50,Harry Bridges...|
|[67,Market at 10t...|[64,2nd at South ...|
|[54,Embarcadero a...|[41,Clay at Batte...|
|[70,San Francisco...|[61,2nd at Townse...|
|[56,Beale at Mark...|[60,Embarcadero a...|
|[73,Grant Avenue ...|[70,San Francisco...|
|[73,Grant Avenue ...|[77,Market at San...|
|[66,South Van Nes...|[51,Embarcadero a...|
|[76,Market at 4th...|[73,Grant Avenue ...|
|[54,Embarcadero a...|[54,Embarcadero a...|
+--------------------+--------------------+
only showing top 10 rows



STATIONS WHERE PEOPLE DID NOT RETURN AFTER LEAVING 

In [173]:
inDeg = g.inDegrees
outDeg = g.outDegrees

In [174]:
degreeRatio = inDeg.join(outDeg, inDeg.id == outDeg.id).drop(outDeg.id).selectExpr("id", "double(inDegree)/double(outDegree) as degreeRatio").cache()

In [177]:
degreeRatio.show()

+--------------------+------------------+
|                  id|       degreeRatio|
+--------------------+------------------+
|Embarcadero at Br...|0.8936255512494988|
|   Market at Sansome|1.2173913043478262|
|       Park at Olive|1.1090425531914894|
|Washington at Kearny|1.3086466165413533|
|Evelyn Park and Ride|0.7413087934560327|
|    San Pedro Square| 1.124823695345557|
|San Antonio Caltr...|0.9886578449905482|
|     2nd at Townsend| 1.102452588050763|
|         MLK Library|0.8735213830755232|
| San Salvador at 1st| 1.105050505050505|
|Civic Center BART...|0.9940721649484536|
|     Spear at Folsom|0.8954072479368497|
|Powell at Post (U...|0.6434241245136186|
|  Powell Street BART| 1.056111397627643|
|South Van Ness at...|0.8303267528931245|
|Stanford in Redwo...|               1.0|
|Arena Green / SAP...|0.9598145285935085|
|      Market at 10th|0.8599074463609592|
|       Market at 4th| 0.978876086517081|
|San Francisco Cal...|1.0351594815699972|
+--------------------+------------

In [191]:
comeback=degreeRatio.join(trips, degreeRatio.id == trips.src).selectExpr("id", "src", "degreeRatio").filter("degreeRatio < 0.9 ")
comeback.show(10)

+--------------------+--------------------+------------------+
|                  id|                 src|       degreeRatio|
+--------------------+--------------------+------------------+
|      Market at 10th|      Market at 10th|0.8599074463609592|
|Embarcadero at Br...|Embarcadero at Br...|0.8936255512494988|
|     Beale at Market|     Beale at Market|  0.75726761574351|
|Grant Avenue at C...|Grant Avenue at C...|0.5180520570948782|
|Grant Avenue at C...|Grant Avenue at C...|0.5180520570948782|
|South Van Ness at...|South Van Ness at...|0.8303267528931245|
|Embarcadero at Br...|Embarcadero at Br...|0.8936255512494988|
|   2nd at South Park|   2nd at South Park|0.8715809483577991|
|     Spear at Folsom|     Spear at Folsom|0.8954072479368497|
|   2nd at South Park|   2nd at South Park|0.8715809483577991|
|Grant Avenue at C...|Grant Avenue at C...|0.5180520570948782|
|Evelyn Park and Ride|Evelyn Park and Ride|0.7413087934560327|
|Embarcadero at Br...|Embarcadero at Br...|0.8936255512

MOTIFS TO FIND A CAN CONNECT B B CAN CONNECT C BUT C AND HAVE NO DIRECT CONNECTIONS

In [ ]:
motifs = g.find("(a)-[e]->(b);(b)-[f]->(c); !(c)-[]->(a)")

motifs.show()

In [ ]:
PAGE RANK ON STATIONS STATION SAN JOSE HAS HIGHEST RANKING

In [11]:
ranks = g.pageRank(resetProbability=0.15, maxIter=5)
ranks.vertices.orderBy(ranks.vertices.pagerank.desc()).limit(20).show()

+----------+--------------------+---------+-----------+---------+-------------+------------+------------------+
|station_id|                  id|      lat|       long|dockcount|     landmark|installation|          pagerank|
+----------+--------------------+---------+-----------+---------+-------------+------------+------------------+
|         2|San Jose Diridon ...|37.329732|-121.901782|       27|     San Jose|    8/6/2013| 2.281525822423689|
|        70|San Francisco Cal...|37.776617| -122.39526|       19|San Francisco|   8/23/2013|1.9076925128811353|
|        22|Redwood City Calt...|37.486078|-122.232089|       25| Redwood City|   8/15/2013|1.4225549210068076|
|        28|Mountain View Cal...|37.394358|-122.076713|       23|Mountain View|   8/15/2013|1.3956850803609109|
|        69|San Francisco Cal...|  37.7766| -122.39547|       23|San Francisco|   8/23/2013|1.2880093979600193|
|        50|Harry Bridges Pla...|37.795392|-122.394203|       23|San Francisco|   8/20/2013|1.0394789374